In [1]:
import os
import tensorflow as tf
import time

In [2]:
def decode(serialized_example):
    features = tf.parse_single_example(
      serialized_example,      
      features={
          'label': tf.FixedLenFeature([], tf.int64),
          'width': tf.FixedLenFeature([], tf.int64),
          'height': tf.FixedLenFeature([], tf.int64),
          'image': tf.FixedLenFeature([], tf.string)
      })
    image = tf.decode_raw(features['image'],tf.uint8)
    #image = tf.reshape(image,[64,64,1])
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [64,64,1])
    label = tf.cast(features['label'],tf.int64)
    
    return image,label

In [3]:
batch_size=100
test_file_name = "Shuffled_PE92_Test.tfrecord"
train_file_name = "Shuffled_PE92_Train.tfrecord"
train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))
print(train_data_length)

test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))
total_test_batch = int(test_data_length/batch_size)+1

200215


In [4]:
#Test Data set
test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode)
#test_dataset = test_dataset.repeat()
#test_dataset= test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)

In [5]:
#Train Data set
train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)

In [6]:
train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,
                                           test_dataset.output_shapes)

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_image, train_label = train_iterator.get_next()
test_image, test_label = test_iterator.get_next()

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

In [7]:
input_images = tf.placeholder(tf.float32,[None,64,64,1])
input_labels = tf.placeholder(tf.int64,[None])

keep_prob = tf.placeholder(tf.float32)
print(input_images)
print(input_labels)

Tensor("Placeholder:0", shape=(?, 64, 64, 1), dtype=float32)
Tensor("Placeholder_1:0", shape=(?,), dtype=int64)


In [8]:
W1 = tf.Variable(tf.random_normal([5,5,1,32], stddev=0.01))
L1 = tf.nn.conv2d(input_images,W1, strides=[1,1,1,1], padding = 'SAME')
#at1 = tf.nn.relu(L1)

B1 = tf.Variable(tf.constant(0.1,shape=[32]))
at1 = tf.nn.relu(L1 + B1)

pool1 =  tf.nn.max_pool(at1, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

print(pool1)

Tensor("MaxPool:0", shape=(?, 32, 32, 32), dtype=float32)


In [9]:
W2 = tf.Variable(tf.random_normal([5, 5, 32, 64], stddev=0.01))

L2 = tf.nn.conv2d(pool1,W2, strides=[1,1,1,1], padding = 'SAME')
#at2 = tf.nn.relu(L2)

B2 = tf.Variable(tf.constant(0.1,shape=[64]))
at2 = tf.nn.relu(L2 + B2)


pool2 =  tf.nn.max_pool(at2, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

print(pool2)

Tensor("MaxPool_1:0", shape=(?, 16, 16, 64), dtype=float32)


In [10]:
W3 = tf.Variable(tf.random_normal([4, 4, 64, 128], stddev=0.01))
B3 = tf.Variable(tf.constant(0.1,shape=[128]))

L3 = tf.nn.conv2d(pool2,W3, strides=[1,1,1,1], padding = 'SAME')
#t3 = tf.nn.relu(L3)
at3 = tf.nn.relu(L3 + B3)

pool3 =  tf.nn.max_pool(at3, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

print(pool3)

Tensor("MaxPool_2:0", shape=(?, 8, 8, 128), dtype=float32)


In [11]:
W4 = tf.Variable(tf.random_normal([4, 4, 128, 256], stddev=0.01))
B3 = tf.Variable(tf.constant(0.1,shape=[256]))

L4 = tf.nn.conv2d(pool3,W4, strides=[1,1,1,1], padding = 'SAME')
#at4 = tf.nn.relu(L4)
at4 = tf.nn.relu(L4 + B3)

pool4 =  tf.nn.max_pool(at4, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

print(pool4)

Tensor("MaxPool_3:0", shape=(?, 4, 4, 256), dtype=float32)


In [12]:
W5 = tf.Variable(tf.random_normal([4*4*256, 512], stddev=0.01))
fc1 = tf.reshape(pool4,[-1,4*4*256])
fb1 = tf.Variable(tf.constant(0.1, shape=[512]))

fc1 = tf.matmul(fc1, W5)
#fc1 = tf.nn.relu(fc1)
fc1 = tf.nn.relu(fc1+fb1)

fc1 = tf.nn.dropout(fc1,keep_prob)
print(fc1)

Tensor("dropout/mul:0", shape=(?, 512), dtype=float32)


In [13]:
W6 = tf.Variable(tf.random_normal([512,2350],stddev=0.01))
fb2 = tf.Variable(tf.constant(0.1,shape=[2350]))
#logit = tf.matmul(fc1, W6)
logit = tf.matmul(fc1, W6)+fb2
print(logit)

Tensor("add_5:0", shape=(?, 2350), dtype=float32)


In [14]:
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit, labels=input_labels))
optimizer = tf.train.AdamOptimizer(0.005).minimize(loss)
loss_summ=tf.summary.scalar("Loss",loss)
print(loss)

Tensor("Mean:0", shape=(), dtype=float32)


In [15]:
correct_prediction = tf.equal(tf.argmax(logit,1),input_labels)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
test_accuracy = tf.reduce_sum(tf.cast(correct_prediction, tf.float32))
acc_summ=tf.summary.scalar("Accuracy",accuracy)

In [17]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:
    
    merged_summary = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter("./logs/Basic/PE92/Train")
    
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(train_init_op)
    sess.run(test_init_op)#
    #Train Model
    
    start_time = time.time()
    for epoch in range(100):
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_labels = sess.run([train_image, train_label])
            sess.run(optimizer,feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:0.5})
            summary, train_acc, train_loss = sess.run([merged_summary, accuracy, loss],feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0})
            
            train_writer.add_summary(summary,i)
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f, train loss: %f' % (epoch,i, train_acc,train_loss))

        
        end_time = time.time()
        elapsed = end_time - start_time
        elapsed = elapsed / 60
        print("Time : %f" %(elapsed))
    
    print("Training is done.")
    #Test Model
    save_path = saver.save(sess, "./save/PE92/Basic/Basic_model_PE92.ckpt")
    acc_sum=0.0
    for epoch in range(total_test_batch):
        batch_images, batch_labels = sess.run([test_image, test_label])
        try:
            test_acc, test_loss=sess.run([test_accuracy,loss], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0})
            test_acc=test_acc/len(batch_labels)
            print("Test acc is : %f"%(test_acc))
        except tf.errors.OutOfRangeError:
              print("Out of Data ierators")
        acc_sum+=test_acc
    test_acc = acc_sum/test_data_length
    print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
    
    coord.request_stop()
    coord.join(threads)
    train_writer.close()
    

Train total Batch :2002
Epoch :0 | step 0, training accuracy: 0.020000, train loss: 451.694092
Epoch :0 | step 100, training accuracy: 0.000000, train loss: 7.768229
Epoch :0 | step 200, training accuracy: 0.000000, train loss: 7.768107
Epoch :0 | step 300, training accuracy: 0.000000, train loss: 7.772088
Epoch :0 | step 400, training accuracy: 0.000000, train loss: 7.781970
Epoch :0 | step 500, training accuracy: 0.000000, train loss: 7.774735
Epoch :0 | step 600, training accuracy: 0.000000, train loss: 7.766427
Epoch :0 | step 700, training accuracy: 0.000000, train loss: 7.775711
Epoch :0 | step 800, training accuracy: 0.000000, train loss: 7.764658
Epoch :0 | step 900, training accuracy: 0.000000, train loss: 7.776766
Epoch :0 | step 1000, training accuracy: 0.000000, train loss: 7.781356
Epoch :0 | step 1100, training accuracy: 0.000000, train loss: 7.784872
Epoch :0 | step 1200, training accuracy: 0.000000, train loss: 7.789337
Epoch :0 | step 1300, training accuracy: 0.000000,

KeyboardInterrupt: 

In [ ]:
train_file_name = "Shuffled_HanDB_Train.tfrecord"
test_file_name = "Shuffled_HanDB_Test.tfrecord"

train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))
test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))
total_test_batch = int(test_data_length/batch_size)+1

train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)


test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode)
test_dataset = test_dataset.repeat()
test_dataset= test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)


train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types, test_dataset.output_shapes)

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_image, train_label = train_iterator.get_next()
test_image, test_label = test_iterator.get_next()

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

In [ ]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:
    
    merged_summary = tf.summary.merge_all()
   
    
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(train_init_op)
    sess.run(test_init_op)#Test iterator 생성
    #Train Model
    loss_sum=0
    start_time = time.time()
    for epoch in range(100):
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_labels = sess.run([train_image, train_label])
            sess.run(optimizer,feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:0.5})
            summary, train_acc, loss=sess.run([merged_summary, accuracy, loss],feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0})
            train_writer.add_summary(summary,i)
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f, Loss: %f' % (epoch,i, train_acc,train_loss))

        save_path = saver.save(sess, "./save/HanDB/Basic/Basic_model_HanDB.ckpt")

        end_time = time.time()
        elapsed = end_time - start_time
        elapsed = elapsed/60
        print("Time : %f" %(elapsed))
        
    print("Training is done.")
    
    acc_sum=0.0
    for epoch in range(total_test_batch):
        batch_images, batch_labels = sess.run([test_image, test_label])
        try:
            test_acc, test_loss=sess.run([test_accuracy,loss], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0})
            test_acc=test_acc/len(batch_labels)
            print("Test acc is : %f"%(test_acc))
        except tf.errors.OutOfRangeError:
              print("Out of Data ierators")
        acc_sum+=test_acc
    test_acc = acc_sum/test_data_length
    print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
    train_writer.close()


In [ ]:
def read_and_decode(filename_queue, batch_size):
    tf_record_reader=tf.TFRecordReader()
    _, tf_record_serialized = tf_record_reader.read(tf_record_filename_queue)

    tf_record_features = tf.parse_single_example(
        tf_record_serialized,
        features={
            'label': tf.FixedLenFeature([], tf.int64),
            'width': tf.FixedLenFeature([], tf.int64),
            'height': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string),
        })
    tf_record_image = tf.decode_raw(tf_record_features['image'],tf.uint8)
    
    tf_record_image = tf.cast(tf_record_image, tf.float32)

    tf_record_image = tf.reshape(tf_record_image, [64,64,1])

    tf_record_label = tf.cast(tf_record_features['label'],tf.int64)
    
    images, labels = tf.train.shuffle_batch([tf_record_image, tf_record_label],
                                         batch_size = batch_size,
                                         capacity=20*batch_size,
                                         min_after_dequeue=5*batch_size )
    return images, labels